In [1]:
from pathlib import Path
import pickle
import numpy as np
import pandas as pd
import os
from multiprocessing import Pool

%load_ext autoreload
%autoreload 2

In [2]:
path_data_dir = Path().cwd().parent.parent / "data"
print(path_data_dir)

folder_raw_data = path_data_dir / "raw"
folder_raw_data_cnc = path_data_dir / "raw/cnc" 

/home/tim/Documents/feat-store/data


In [3]:
files = os.listdir(folder_raw_data_cnc)
file_list = [
    Path(folder_raw_data_cnc) / filename
    for filename in files
    if filename.endswith(".pickle")
]

In [6]:
col_selection = ["current_main", "current_sub", "tool_no"]

def read_pickle(filename):
    'converts a filename to a pandas dataframe'
    df = pickle.load(open(filename, 'rb'))
    id = filename.stem
    df["id"] = id
    df['timestamp'] = id.split("_")[0]
    df['index_no'] = id.split("_")[-1]
    
    return df[col_selection]
    

def main(folder_interim_data):
    """Runs data processing scripts to turn raw data from (../raw) into
    cleaned data ready to be analyzed (saved in ../processed).
    """

    # get a list of file names
    files = os.listdir(folder_interim_data)
    file_list = [
        Path(folder_interim_data) / filename
        for filename in files
        if filename.endswith(".pickle")
    ]

    # set up your pool
    with Pool(processes=2) as pool:  # or whatever your hardware can support

        # have your pool map the file names to dataframes
        df_list = pool.map(read_pickle, file_list)

        # reduce the list of dataframes to a single dataframe
        combined_df = pd.concat(df_list, ignore_index=True)

        return combined_df

In [7]:
if __name__ == "__main__":
    df = main(folder_raw_data_cnc)
    print("Final df shape:", df.shape)

Final df shape: (38416, 3)


In [8]:
df.head()

,current_main,current_sub,tool_no
0,1,825,54
1,3,806,54
2,1,802,54
3,2,759,54
4,2,718,54


In [16]:
df = read_pickle(file_list[0])
df.head()

,cut_signal,current_main,current_sub,power_main,power_sub,error_x,error_z,speed_main,speed_sub,tool_no
27432,1,1,825,0.007324,2.659912,0.0005,-0.385,0,-5002,54
27433,1,3,806,0.009766,2.680664,0.0005,-0.374,0,-5001,54
27434,1,1,802,0.020752,2.801514,0.0005,-0.364,0,-4998,54
27435,1,2,759,0.024414,2.807617,0.0005,-0.353,0,-5001,54
27436,1,2,718,0.024414,2.827148,0.0005,-0.343,0,-5000,54


In [17]:
file_list[0].stem

'1540309555_54_5'

In [15]:
df['cut_signal'].unique()

array([1], dtype=int16)